In [1]:
import serial
import time
import pandas as pd
import plotly.express as px


In [3]:
ser = serial.Serial('COM9', 115200, timeout=1)

In [4]:
def readToBuffer():
    ser.reset_input_buffer()
    buffer = []
    while True:
        data = ser.readline().decode('utf-8').strip()
        # print(data)
        if data:
            buffer.append(data)
            if len(buffer) > 100:
                break
        else:
            time.sleep(0.1)
    return buffer
        
def convertToDistance(raw_data):
    if type(raw_data) == str:
        raw_data = int(raw_data)
    return raw_data * 0.017

def plotDistance(label):
    df = pd.DataFrame(readToBuffer(), columns=['raw_data'])
    df['distance[cm]'] = df['raw_data'].apply(convertToDistance)


    fig = px.line(df, y='distance[cm]', title=label)
    fig.show()

    fig2 = px.histogram(df, x='distance[cm]', title=label)
    fig2.show()
    df.to_csv(label + '.csv')

In [5]:
plotDistance('100cm')

In [30]:
plotDistance('90cm')

In [31]:
plotDistance('80cm')

In [32]:
plotDistance('70cm')

In [33]:
plotDistance('60cm')

In [36]:
plotDistance('50cm13diagonal')

In [35]:
plotDistance('40cm')

In [36]:
plotDistance('30cm')

In [37]:
plotDistance('20cm')

In [38]:
plotDistance('10cm')

In [39]:
plotDistance('1cm')

In [40]:
plotDistance('0cm')

In [54]:
#mean value of each distance
import os, sys

df = pd.DataFrame(columns=['distance[cm]', 'intended_distance[cm]'])

files = os.listdir()
for file in files:
    if file.endswith('.csv'):
        intended_distance = file.split('.')[0].split('cm')[0]
        intended_distance = int(intended_distance)
        if intended_distance == 0:
            continue
        df2 = pd.read_csv(file)
        distance = df2['distance[cm]'].mean()
        df = pd.concat([df, pd.DataFrame([[distance.mean(), intended_distance]], columns=['distance[cm]', 'intended_distance[cm]'])])

df = df.sort_values(by=['intended_distance[cm]'])

#sort the df by the value of intended_distance+

fig = px.line(df, x='intended_distance[cm]', y='distance[cm]')
fig.update_layout(title='Linearity of the sensor')
fig.show()


In [61]:
#mean value of each distance
import os, sys

df = pd.DataFrame(columns=['microseconds', 'tof distance[cm]'])

files = os.listdir()
for file in files:
    if file.endswith('.csv'):
        intended_distance = file.split('.')[0].split('cm')[0]
        intended_distance = int(intended_distance)
        if intended_distance == 0:
            continue
        df2 = pd.read_csv(file)
        df2['microseconds'] = df2['raw_data'].apply(lambda x: int(x))
        microseconds = df2['microseconds'].mean()
        
        df = pd.concat([df, pd.DataFrame([[microseconds, intended_distance*2]], columns=['microseconds', 'tof distance[cm]'])])
display(df)

,microseconds,tof distance[cm]
0,5704.900990,200
0,512.009901,20
0,246.267327,2
0,1096.336634,40
0,1667.257426,60
0,2247.881188,80
0,2830.326733,100
0,3397.316832,120
0,3982.425743,140
0,4550.277228,160


In [70]:
fig = px.line(df, x='tof distance[cm]', y='microseconds')
fig.update_layout(title='Linearity of the sensor')
fig.show()

In [79]:
#speed of sound should be then distance / time
df_speed = pd.DataFrame(columns=['tof distance[cm]', 'speed of sound[m/s]'])

for i in range(1, len(df)):
    speed = df.iloc[i]['tof distance[cm]'] / df.iloc[i]['microseconds'] 
    speed = speed * 10000
    df_speed = pd.concat([df_speed, pd.DataFrame([[df.iloc[i]['tof distance[cm]'], speed]], columns=['tof distance[cm]', 'speed of sound[m/s]'])])
    
display(df_speed)

fig = px.line(df_speed, x='tof distance[cm]', y='speed of sound[m/s]')
fig.update_layout(title='Speed of sound vs TOF distance')

,tof distance[cm],speed of sound[m/s]
0,20,390.617446
0,2,81.212560
0,40,364.851440
0,60,359.872441
0,80,355.890696
0,100,353.316099
0,120,353.219926
0,140,351.544534
0,160,351.626927
0,180,351.631858


In [80]:
ser.close()

In [89]:
plotDistance('50cmdiagonal')

ValueError: invalid literal for int() with base 10: '103.87'

In [37]:
set_acceleration = 0b00011000
print(hex(set_acceleration))

0x18
